Retrieving notices: ...working... done
Channels:
 - defaults
Platform: osx-arm64
Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda3

  added / updated specs:
    - scikit-learn


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2024.7.2   |       hca03da5_0         128 KB
    certifi-2024.8.30          |  py312hca03da5_0         164 KB
    conda-24.7.1               |  py312hca03da5_0         1.2 MB
    openssl-3.0.15             |       h80987f9_0         4.3 MB
    scikit-learn-1.5.1         |  py312hd77ebd4_0         9.7 MB
    threadpoolctl-3.5.0        |  py312h989b03a_0          49 KB
    ------------------------------------------------------------
                                           Total:        15.5 MB

The following packages will be UPDATED:

  ca-certificates                      2024.3.11-hca03da5_0 --> 2024.7.2-hca03da

In [1]:
from tensorflow import keras
import tensorflow as tf
import matplotlib as mlt
import numpy as np
import pandas as pd

x_set = []
y_set = []

with open("dialog_acts.dat", 'r') as file:
    for line in file:
        y_set.append(line.split()[0])
        x_set.append(" ".join(line.split()[1:]).lower())
    # dialog_dataset = file.read()

y_set[0], x_set[0]
    

('inform', 'im looking for a moderately priced restaurant that serves')

In [14]:
x_train, x_test = x_set[int(len(x_set)*0.15):], x_set[:int(len(x_set)*0.15)]
y_train, y_test = y_set[int(len(y_set)*0.15):], y_set[:int(len(y_set)*0.15)]
len(x_train), len(x_test), len(y_train), len(y_test)

y_train = pd.Series(y_train)
y_train = pd.factorize(y_train)[0]
y_train_cat = keras.utils.to_categorical(y_train, 15)
encoder = keras.layers.CategoryEncoding(num_tokens=15, output_mode="one_hot")
y_train_cat = encoder(y_train)
print(y_train_cat.numpy())
x_train = np.asarray(x_train)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [25]:
from gensim.models.doc2vec import Doc2Vec,\
    TaggedDocument
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\karst\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [33]:
#Doc2Vec (Sentences to vector)

tagged_x_data = [TaggedDocument(words=word_tokenize(doc.lower()), tags=[str(i)]) for i, doc in enumerate(x_train)]
model = Doc2Vec(vector_size=15,
                min_count=2, epochs=50)
print(tagged_x_data)
model.build_vocab(tagged_x_data)
model.train(tagged_x_data,
            total_examples=model.corpus_count,
            epochs=model.epochs)

document_vectors = [model.infer_vector(
    word_tokenize(doc.lower())) for doc in x_train]
 

[TaggedDocument(words=['how', 'bout', 'thai', 'food'], tags=['0']), TaggedDocument(words=['yes'], tags=['1']), TaggedDocument(words=['noise'], tags=['2']), TaggedDocument(words=['noise'], tags=['3']), TaggedDocument(words=['thai', 'food'], tags=['4']), TaggedDocument(words=['dont', 'care'], tags=['5']), TaggedDocument(words=['what', 'is', 'the', 'addre'], tags=['6']), TaggedDocument(words=['what', 'is', 'the', 'phone', 'number'], tags=['7']), TaggedDocument(words=['thank', 'you', 'goodbye'], tags=['8']), TaggedDocument(words=['expensive'], tags=['9']), TaggedDocument(words=['south'], tags=['10']), TaggedDocument(words=['south'], tags=['11']), TaggedDocument(words=['south'], tags=['12']), TaggedDocument(words=['any'], tags=['13']), TaggedDocument(words=['expensive'], tags=['14']), TaggedDocument(words=['is', 'there', 'anything', 'else'], tags=['15']), TaggedDocument(words=['whats', 'the', 'address'], tags=['16']), TaggedDocument(words=['thank', 'you', 'good', 'bye'], tags=['17']), Tagge

In [43]:

model = keras.Sequential()
model.add(keras.Input(shape=(15,)))
model.add(keras.layers.Dense(2, activation="relu"))
#Last layer has dimension 15
model.add(keras.layers.Dense(15, activation="softmax"))
model.summary()

model.compile(loss='categorical_crossentropy',
optimizer=keras.optimizers.RMSprop(), metrics=['accuracy'])
history = model.fit(document_vectors, y_train_cat, batch_size=256,
epochs=20, verbose=1)

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_22 (Dense)                │ (None, 2)              │            32 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 15)             │            45 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 77 (308.00 B)

 Trainable params: 77 (308.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 935us/step - accuracy: 0.2104 - loss: 2.6550
Epoch 2/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5457 - loss: 2.4735
Epoch 3/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 993us/step - accuracy: 0.5689 - loss: 2.2676
Epoch 4/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 981us/step - accuracy: 0.5573 - loss: 2.0489
Epoch 5/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 958us/step - accuracy: 0.5234 - loss: 1.8450
Epoch 6/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 982us/step - accuracy: 0.5133 - loss: 1.6798
Epoch 7/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 952us/step - accuracy: 0.5511 - loss: 1.5419
Epoch 8/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 972us/step - accuracy: 0.5741 - loss: 1.4555
Epoch 9/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 929us/step - accuracy: 0.5921 - loss: 1.3650
Epoch 10/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5951 - loss: 1.3039
Epoch 11/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6090 - loss: 1.2380
Epoch 12/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accur